In [1]:
! pip install transformers datasets tensorflow

  Using cached pyarrow-22.0.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
Using cached pyarrow-22.0.0-cp311-cp311-win_amd64.whl (28.1 MB)

  Attempting uninstall: pyarrow

    Found existing installation: pyarrow 20.0.0

   ---------------------------------------- 0/4 [pyarrow]
    Uninstalling pyarrow-20.0.0:
   ---------------------------------------- 0/4 [pyarrow]
      Successfully uninstalled pyarrow-20.0.0
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   -

In [2]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.summary() # This will show the architecture layers in your notebook

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [3]:
# Prepare data
texts = ["This movie was fantastic!", "I did not like this film."]
labels = [1, 0]

# Tokenize inputs
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="tf")

# Convert labels to tensor
targets = tf.convert_to_tensor(labels)

In [4]:
# Fine-tuning configuration
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

# Run training
history = model.fit(
    inputs.data, 
    targets, 
    epochs=3, 
    batch_size=2
)

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



1/1 [==============================] - 52s 52s/step - loss: 0.8365 - accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 1s 678ms/step - loss: 0.7357 - accuracy: 0.5000
Epoch 3/3
1/1 [==============================] - 1s 685ms/step - loss: 0.5512 - accuracy: 0.5000


In [5]:
def predict_sentiment(text):
    predict_input = tokenizer(text, padding=True, truncation=True, return_tensors="tf")
    output = model.predict(predict_input.data)
    prediction_value = tf.argmax(output.logits, axis=1).numpy()[0]
    
    sentiment = "Positive" if prediction_value == 1 else "Negative"
    return sentiment

# Test it out
print(f"Result: {predict_sentiment('The acting was incredible.')}")

1/1 [==============================] - 8s 8s/step
Result: Negative
